In [1]:
import pandas as pd
import pyomo.environ as pyo
from collections import defaultdict
from pathlib import Path

# conda install -c conda-forge coincbc
# conda install -c conda-forge ipopt

In [2]:
def data_ingest(data_path: Path, year: int):
    counties = pd.read_csv(data_path / "oh_county_list.csv")
    counties["county_id"] = pd.Series(range(1, 89))

    df_pop = pd.read_csv(data_path / f"oh_county_pop_{str(year)}.csv")
    df = pd.merge(counties, df_pop, on="county")

    return df, counties


def create_adjacent_list(data_path: Path) -> list:
    with open(data_path / "oh_adjacent_loc.dat", "r") as f:
        read_adjacent = f.readlines()

    adjacent_sublist = list(map(lambda x: x.strip("\n").split(","), read_adjacent))
    adjacent_list = [[int(x) for x in sublst] for sublst in adjacent_sublist]

    return adjacent_list

data_path = Path("data")
year = 2021

adjacent_list = create_adjacent_list(data_path)
df_init, counties = data_ingest(data_path, year)

In [3]:
adjacent_matrix = defaultdict(list)
for i in range(1, 89):
    adjacent_list[i - 1].append(i)
    adjacent_matrix[i] = adjacent_list[i - 1]
# adj_matrix

[8, 36, 66, 73, 1]
[6, 32, 33, 69, 81, 2]
[38, 39, 42, 47, 52, 70, 85, 3]
[28, 43, 78, 4]
[37, 53, 58, 64, 82, 84, 5]
[2, 19, 33, 46, 54, 75, 81, 6]
[30, 34, 41, 56, 61, 7]
[1, 13, 14, 36, 8]
[31, 57, 68, 83, 9]
[15, 34, 41, 76, 79, 10]
[12, 46, 49, 55, 75, 80, 11]
[11, 29, 49, 55, 57, 12]
[8, 14, 31, 83, 13]
[8, 13, 24, 29, 36, 83, 14]
[10, 41, 50, 76, 15]
[30, 38, 42, 45, 60, 79, 16]
[39, 51, 59, 70, 74, 88, 17]
[28, 43, 47, 52, 77, 18]
[6, 54, 55, 57, 68, 75, 19]
[35, 63, 69, 86, 20]
[25, 42, 45, 51, 59, 80, 21]
[39, 47, 72, 22]
[25, 37, 45, 64, 65, 23]
[14, 29, 36, 49, 65, 71, 24]
[21, 23, 45, 49, 65, 80, 25]
[35, 48, 86, 26]
[36, 40, 44, 53, 82, 27]
[4, 18, 43, 67, 78, 28]
[12, 14, 24, 49, 57, 83, 29]
[7, 16, 34, 60, 61, 79, 30]
[9, 13, 83, 31]
[2, 33, 69, 74, 87, 88, 32]
[2, 6, 32, 46, 51, 80, 88, 33]
[7, 10, 30, 41, 79, 34]
[20, 26, 48, 69, 86, 87, 35]
[1, 8, 14, 24, 27, 66, 71, 36]
[5, 23, 64, 65, 71, 82, 37]
[3, 16, 42, 76, 79, 85, 38]
[3, 17, 22, 47, 70, 72, 74, 39]
[27, 44, 

In [4]:
def county_list_to_names(id_list, counties):
    return counties[counties["county_id"].isin(id_list)]["county"].tolist()


df["adj_names"] = df["adj_id"].apply(lambda x: county_list_to_names(x, counties))

NameError: name 'df' is not defined

In [ ]:
df

In [ ]:
adj_matrix[88]

In [ ]:
model = pyo.ConcreteModel()

# declare decision variables
model.x = pyo.Var(domain=pyo.NonNegativeReals)

# declare objective
model.profit = pyo.Objective(
    expr = 40*model.x,
    sense = pyo.maximize)

# declare constraints
model.demand = pyo.Constraint(expr = model.x <= 40)
model.laborA = pyo.Constraint(expr = model.x <= 80)
model.laborB = pyo.Constraint(expr = 2*model.x <= 100)

# solve
pyo.SolverFactory('cbc').solve(model).write()

model.profit.display()
model.x.display()


In [ ]:
def param_adj(m, i, j):
    return int(j in adj_matrix[i])


def param_pop(m, i):
    return df["population"][i - 1]

In [ ]:
int(20 in adj_matrix[25])
df["population"][24 - 1]

In [ ]:
model = pyo.AbstractModel()

# value of n (number of counties)
model.n = pyo.Param(initialize=88)

# range of i and j (iterating over counties)
model.I = pyo.RangeSet(1, model.n)
model.J = pyo.RangeSet(1, model.n)

# TOSET limit on number of pirincipal places of buisnesses opened (init to 5)
model.k = pyo.Var(within=pyo.NonNegativeIntegers, initialize=5)

model.p = pyo.Param(model.I, initialize=param_pop)  # population of county i
model.a = pyo.Var(
    model.I, model.J, domain=pyo.Binary, initialize=param_adj
)  # 1 if county i and j are adjacent
model.x = pyo.Var(
    model.J, domain=pyo.Binary
)  # 1 if principal place of business is opened in county j
model.y = pyo.Var(model.I, domain=pyo.Binary)  # 1 if county i is covered

In [ ]:
def obj_sum(m):
    return pyo.summation(m.p, m.y)


def con_a(m, i):
    return sum((m.a[i, j] * m.x[j]) for j in m.J) >= m.y[i]


def con_x(m):
    return sum(m.x[j] for j in m.J) <= m.k

In [ ]:
model.obj = pyo.Objective(rule=obj_sum, sense=pyo.maximize)

model.a_constraint = pyo.Constraint(model.I, rule=con_a)
model.x_constraint = pyo.Constraint(rule=con_x)

In [ ]:
# data = {None: {
#     'I': {None: [1,2,3]},
#     'p': {None: 100},
#     'q': {1: 10, 2:20, 3:30},
#     'r': {(1,1): 110, (1,2): 120, (2,3): 230},
# }}
instance = model.create_instance()

In [ ]:
# instance.pprint()()

In [ ]:
opt = pyo.SolverFactory("cbc")
opt.solve(instance)